**Necessary library**

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Load the dataset**
**copy link from raw helpful link:https://medium.com/towards-entrepreneurship/importing-a-csv-file-from-github-in-a-jupyter-notebook-e2c28e7e74a5**

In [9]:

url = 'https://raw.githubusercontent.com/NNishat/Nishat_files/master/dataset%20-%20ques.csv'
df = pd.read_csv(url)
# Dataset is now stored in a Pandas Dataframe

In [10]:
df.head(5)

,Question,Attributes of God
0,How is Adam different?,0
1,What does Allah know?,1
2,What does Allah know about Adam?,1
3,Who owns heaven and earth?,1
4,Who helps and protects mankind?,1


**Define independent and dependent features**

In [11]:
X= df['Question']
y= df['Attributes of God']

**Copy the dataframe into mesages variable**

In [12]:
mesages= df.copy()

In [13]:
mesages.head(4)

,Question,Attributes of God
0,How is Adam different?,0
1,What does Allah know?,1
2,What does Allah know about Adam?,1
3,Who owns heaven and earth?,1


**Apply RESET Index on the dataframe**

In [14]:
mesages.reset_index(inplace=True)

**Library import**

In [15]:
import re
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

**Text preprocessing**

In [16]:
#Ideas from this Youtube link about Naive bayes and bag of words--------------------https://www.youtube.com/watch?v=8Mlc4-3tgzc
#github link--------------https://github.com/Suji04/NormalizedNerd/blob/master/Introduction%20to%20NLP/Bag%20of%20Words.ipynb
import nltk
nltk.download('punkt')
nltk.download('stopwords')

def clean_text(df):
    all_reviews = list()
    lines = df["Question"].values.tolist()
    for text in lines:
        text = text.lower()
        #removing all the links
        pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
        text = pattern.sub('', text)
        #removing punctuation
        text = re.sub(r"[,.\"!@#$%^&*(){}?/;`~:<>+=-]", "", text)
        tokens = word_tokenize(text)
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        stop_words.discard("not")
        PS = PorterStemmer()
        #words = [w for w in words if not w in stop_words]
        words = [PS.stem(w) for w in words if not w in stop_words]
        words = ' '.join(words)
        all_reviews.append(words)
    return all_reviews

all_reviews = clean_text(df)
all_reviews[0:10]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['adam differ',
 'allah know',
 'allah know adam',
 'own heaven earth',
 'help protect mankind',
 'origin heaven earth',
 'decid matter',
 'allah creat',
 'bless allah given',
 'allah want human bless']

**Features extraction using CountVectorizer**

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
CV = CountVectorizer(min_df=3)   
X = CV.fit_transform(all_reviews).toarray()
y = mesages["Attributes of God"]

In [18]:
X.shape

(73, 16)

In [27]:
#printing first 5 rows https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/
df.loc[0:15]

,Question,Attributes of God
0,How is Adam different?,0
1,What does Allah know?,1
2,What does Allah know about Adam?,1
3,Who owns heaven and earth?,1
4,Who helps and protects mankind?,1
5,Who originated the heaven and earth?,1
6,Who decides all matters?,1
7,How does Allah create?,1
8,What blessings has Allah given?,0
9,What does Allah want humans to do with blessings?,0


In [28]:
df['Question'][9]

'What does Allah want humans to do with blessings?'

**Train test split**

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 8)

In [22]:
# from sklearn.tree import DecisionTreeClassifier
# model = DecisionTreeClassifier(criterion="entropy", random_state=41)
#explaining random state https://www.youtube.com/watch?v=c249O4giblM
#cross validation https://www.youtube.com/watch?v=fKz-SgScM3Q

#from sklearn.naive_bayes import MultinomialNB
#model = MultinomialNB()
#from sklearn.naive_bayes import BernoulliNB
#model = BernoulliNB()
#from sklearn.naive_bayes import svm
#model=svm.SVC()

**GaussianNB Algorithm**

In [23]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, precision_score
model = GaussianNB()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print("SCORE:",accuracy_score(y_test, y_pred))
print("F1-SCORE:",f1_score(y_test, y_pred))
print("PRECISION:",precision_score(y_test, y_pred))

SCORE: 0.6
F1-SCORE: 0.7000000000000001
PRECISION: 0.6363636363636364


**Predict manually**

In [24]:
model.predict(X_test)

array([0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1])

**K-fold cross validiation**

In [25]:
from sklearn.model_selection import cross_validate
cv_gaussianNB= cross_validate(model, X, y, cv=10)
print(cv_gaussianNB['test_score'])
print(cv_gaussianNB['test_score'].mean())

[0.875      0.75       0.625      0.42857143 0.57142857 0.28571429
 0.71428571 0.57142857 0.71428571 0.42857143]
0.5964285714285714


**Predict from unknown dataset**

In [35]:
Questest = [
          
    
    "Who decides all matters?",

    "Am I crazy?",

    "Are machines crazy?",
    
    "What does Allah want humans to do with blessings?",
]
Questest_count = CV.transform(Questest).toarray()
model.predict(Questest_count)

array([1, 1, 1, 0])